# [Quantum Phase Estimation Code used from IBM's Qiskit Tutorials](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html)
## Code adapted for Cirq

In [2]:
import cirq
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def qft_dagger_cirq(qc, qubits, n):
    for qubit in range(n//2):
        qc.append(cirq.SWAP(qubits[qubit], qubits[n-qubit-1]))
    for j in range(n):
        for m in range(j):
            qc.append((cirq.CZ**(-1/2**(j-m)))(qubits[m],qubits[j]))
        qc.append(cirq.H(qubits[j]))

In [4]:
def generalised_qpe_cirq(amt_estimation_qubits, angle, shots = 4098):
    # Create and set up circuit
    qubits = cirq.LineQubit.range(amt_estimation_qubits+1)
    circuit = cirq.Circuit()
    
    # Apply H-Gates to counting qubits:
    for qubit in range(amt_estimation_qubits):
        circuit.append(cirq.H(qubits[qubit]))
        
    # Prepare our eigenstate |psi>:
    circuit.append(cirq.X(qubits[amt_estimation_qubits]))
    
    repetitions = 1
    for counting_qubit in range(amt_estimation_qubits):
        for i in range(repetitions):
            circuit.append((cirq.CZ**(angle))(qubits[counting_qubit],qubits[amt_estimation_qubits]));
        repetitions *= 2
        
    # Do the inverse QFT:
    qft_dagger_cirq(circuit, qubits, amt_estimation_qubits)
    
    # Measure of course!
    circuit.append(cirq.measure(*qubits[:-1], key='m'))
        
    simulator = cirq.Simulator()
    results = simulator.run(circuit , repetitions = shots)
    theta_estimates = np.sum(2 ** np.arange(amt_estimation_qubits) * results.measurements['m'], axis=1) / 2**amt_estimation_qubits
    unique,pos = np.unique(theta_estimates,return_inverse=True)
    counts = np.bincount(pos)
    maxpos = counts.argmax()
    
    
generalised_qpe_cirq(8,(2*(1/3)))

# Benchmarking speed of Cirq for QPE

First we modify the previous functions to remove prints, then run it 100,000 times

In [5]:
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)
%timeit -p 8 -r 1 -n 1 generalised_qpe_cirq(8,(2*(1/3)),10000)

129.8528 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
130.8356 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
140.4431 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
137.3588 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
126.2163 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
132.2742 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
131.901 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
130.6857 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
125.8239 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
127.9589 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
